<a href="https://colab.research.google.com/github/7201krap/PYTORCH_project/blob/main/sparsity_4_HL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torch.utils.data import DataLoader, Subset
from torchvision import transforms
import numpy as np
from google.colab import drive

import os
import subprocess as sp
from torchvision.datasets.mnist import MNIST, read_image_file, read_label_file
from torchvision.datasets.utils import extract_archive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.manual_seed(1234)
np.random.seed(1234)

In [ ]:
def patched_download(self):
    """wget patched download method.
    """
    if self._check_exists():
        return

    os.makedirs(self.raw_folder, exist_ok=True)
    os.makedirs(self.processed_folder, exist_ok=True)

    # download files
    for url, md5 in self.resources:
        filename = url.rpartition('/')[2]
        download_root = os.path.expanduser(self.raw_folder)
        extract_root = None
        remove_finished = False

        if extract_root is None:
            extract_root = download_root
        if not filename:
            filename = os.path.basename(url)
        
        # Use wget to download archives
        sp.run(["wget", url, "-P", download_root])

        archive = os.path.join(download_root, filename)
        print("Extracting {} to {}".format(archive, extract_root))
        extract_archive(archive, extract_root, remove_finished)

    # process and save as torch files
    print('Processing...')

    training_set = (
        read_image_file(os.path.join(self.raw_folder, 'train-images-idx3-ubyte')),
        read_label_file(os.path.join(self.raw_folder, 'train-labels-idx1-ubyte'))
    )
    test_set = (
        read_image_file(os.path.join(self.raw_folder, 't10k-images-idx3-ubyte')),
        read_label_file(os.path.join(self.raw_folder, 't10k-labels-idx1-ubyte'))
    )
    with open(os.path.join(self.processed_folder, self.training_file), 'wb') as f:
        torch.save(training_set, f)
    with open(os.path.join(self.processed_folder, self.test_file), 'wb') as f:
        torch.save(test_set, f)

    print('Done!')


MNIST.download = patched_download

In [ ]:
mnist_trainset = MNIST(root='./data', train=True, 
                                download=True, 
                                transform=transforms.Compose([transforms.ToTensor()]))

mnist_testset  = MNIST(root='./data', 
                                train=False, 
                                download=True, 
                                transform=transforms.Compose([transforms.ToTensor()]))

train_dataloader = torch.utils.data.DataLoader(mnist_trainset, 
                                               batch_size=50, 
                                               shuffle=True)

test_dataloader  = torch.utils.data.DataLoader(mnist_testset, 
                                               batch_size=50, 
                                               shuffle=False)

print("Training dataset size: ", len(mnist_trainset))
print("Testing dataset size: ",  len(mnist_testset))

Training dataset size:  60000
Testing dataset size:  10000


In [ ]:
# ************* modify this section for later use *************
# Define the model 
class Model(torch.nn.Module):
    def __init__(self):
        super(Model, self).__init__()

        # modify this section for later use 
        self.linear_1 = torch.nn.Linear(784, 256)
        self.linear_2 = torch.nn.Linear(256, 256)
        self.linear_3 = torch.nn.Linear(256, 256)
        self.linear_4 = torch.nn.Linear(256, 256)
        self.linear_5 = torch.nn.Linear(256, 10)
        self.sigmoid12  = torch.nn.Sigmoid()
        self.sigmoid23  = torch.nn.Sigmoid()
        self.sigmoid34  = torch.nn.Sigmoid()
        self.sigmoid45  = torch.nn.Sigmoid()

        self.layer_activations = dict()

    def forward(self, x):

        # modify this section for later use 
        x = x.reshape(x.size(0), -1)
        x = self.linear_1(x)
        x = self.sigmoid12(x)
        x = self.linear_2(x)
        x = self.sigmoid23(x)
        x = self.linear_3(x)
        x = self.sigmoid34(x)
        x = self.linear_4(x)
        x = self.sigmoid45(x)
        pred = self.linear_5(x)
        return pred
# ************* modify this section for later use *************

In [ ]:
def get_activation(model, layer_name):    
    def hook(module, input, output):
        model.layer_activations[layer_name] = output
    return hook

In [ ]:
def sparsity_calculator(final_spareness):
    sparseness_list = list()
    for single_epoch_spareness in final_spareness:

        hidden_layer_activation_list = single_epoch_spareness
        hidden_layer_activation_list = torch.stack(hidden_layer_activation_list)
        layer_activations_list = torch.reshape(hidden_layer_activation_list, (10000, 256))

        layer_activations_list = torch.abs(layer_activations_list)  # modified 
        num_neurons = layer_activations_list.shape[1]
        population_sparseness = (np.sqrt(num_neurons) - (torch.sum(layer_activations_list, dim=1) / torch.sqrt(torch.sum(layer_activations_list ** 2, dim=1)))) / (np.sqrt(num_neurons) - 1)
        mean_sparseness_per_epoch = torch.mean(population_sparseness)

        sparseness_list.append(mean_sparseness_per_epoch)

    return sparseness_list

In [ ]:
def model_factory(optimizer_name):
    '''
    optimizer_name : choose one of Adagrad, Adadelta, SGD, and Adam 

    '''
    my_model = Model()
    print("my_model:", my_model)
    my_model.to(device)

    # ************* modify this section for later use *************
    # modify this section for later use 
    my_model.sigmoid12.register_forward_hook(get_activation(my_model, 's12'))
    my_model.sigmoid23.register_forward_hook(get_activation(my_model, 's23'))
    my_model.sigmoid34.register_forward_hook(get_activation(my_model, 's34'))
    my_model.sigmoid45.register_forward_hook(get_activation(my_model, 's45'))
    # ************* modify this section for later use *************

    if optimizer_name == 'Adadelta':
        my_optimizer = torch.optim.Adadelta(my_model.parameters(), lr=1.0)

    elif optimizer_name == 'Adagrad':
        my_optimizer = torch.optim.Adagrad(my_model.parameters(), lr=0.1)

    elif optimizer_name == 'SGD':
        my_optimizer = torch.optim.SGD(my_model.parameters(), lr=0.1)

    elif optimizer_name == 'Adam':
        my_optimizer = torch.optim.Adam(my_model.parameters(), lr=0.001)

    else:
        print("ERROR")
    
    print("my_optimizer:", my_optimizer)
    test_acc, sparseness_list, spar12, spar23, spar34, spar45 = sparsity_trainer(optimizer=my_optimizer, model=my_model)
    # ************* modify this section for later use *************
    file_saver = open(f"4HL_sparsity_new_{optimizer_name}.txt", "w")
    # ************* modify this section for later use *************
    file_saver.write(str(test_acc)+'\n'+str(sparseness_list)+'\n'+str(spar12)+'\n'+str(spar23)+'\n'+str(spar34)+'\n'+str(spar45)+'\n\n')
    file_saver.close()

    # ************* modify this section for later use *************
    if optimizer_name == 'Adadelta':
        !cp 4HL_sparsity_new_Adadelta.txt /content/drive/MyDrive
    
    elif optimizer_name == 'Adagrad':
        !cp 4HL_sparsity_new_Adagrad.txt /content/drive/MyDrive

    elif optimizer_name == 'SGD':
        !cp 4HL_sparsity_new_SGD.txt /content/drive/MyDrive

    elif optimizer_name == 'Adam':
        !cp 4HL_sparsity_new_Adam.txt /content/drive/MyDrive
    # ************* modify this section for later use *************

    else:
        print("ERROR")

In [ ]:
def sparsity_trainer(optimizer, model):
    criterion = torch.nn.CrossEntropyLoss()
    no_epochs = 50

    train_loss = list()
    test_loss  = list()
    test_acc   = list()

    # ************* modify this section for later use *************
    final_spareness_12 = list()
    final_spareness_23 = list()
    final_spareness_34 = list()
    final_spareness_45 = list()
    # ************* modify this section for later use *************

    best_test_loss = 1
    for epoch in range(no_epochs):
        total_train_loss = 0
        total_test_loss = 0

        # ************* modify this section for later use *************
        hidden_layer_activation_list_12 = list()
        hidden_layer_activation_list_23 = list()
        hidden_layer_activation_list_34 = list()
        hidden_layer_activation_list_45 = list()
        # ************* modify this section for later use *************

        # training
        # set up training mode 
        model.train()
        for itr, (image, label) in enumerate(train_dataloader):
            image, label = image.to(device), label.to(device)

            optimizer.zero_grad()

            pred = model(image)

            loss = criterion(pred, label)
            total_train_loss += loss.item()

            loss.backward()
            optimizer.step()

        total_train_loss = total_train_loss / (itr + 1)
        train_loss.append(total_train_loss)

        # testing 
        # change to evaluation mode 
        model.eval()
        total = 0
        for itr, (image, label) in enumerate(test_dataloader):
            image, label = image.to(device), label.to(device)

            pred = model(image)

            loss = criterion(pred, label)
            total_test_loss += loss.item()

            # we now need softmax because we are testing.
            pred = torch.nn.functional.softmax(pred, dim=1)
            for i, p in enumerate(pred):
                if label[i] == torch.max(p.data, 0)[1]:
                    total = total + 1

            # ***************** sparsity calculation ***************** #
            hidden_layer_activation_list_12.append(model.layer_activations['s12'])
            hidden_layer_activation_list_23.append(model.layer_activations['s23'])
            hidden_layer_activation_list_34.append(model.layer_activations['s34'])
            hidden_layer_activation_list_45.append(model.layer_activations['s45'])

        # this conains activations for all epochs 
        final_spareness_12.append(hidden_layer_activation_list_12)
        final_spareness_23.append(hidden_layer_activation_list_23)
        final_spareness_34.append(hidden_layer_activation_list_34)
        final_spareness_45.append(hidden_layer_activation_list_45)
        # ***************** sparsity calculation ***************** #

        # caculate accuracy 
        accuracy = total / len(mnist_testset)

        # append accuracy here
        test_acc.append(accuracy)

        # append test loss here 
        total_test_loss = total_test_loss / (itr + 1)
        test_loss.append(total_test_loss)

        print('\nEpoch: {}/{}, Train Loss: {:.8f}, Test Loss: {:.8f}, Test Accuracy: {:.8f}'.format(epoch + 1, no_epochs, total_train_loss, total_test_loss, accuracy))

        if total_test_loss < best_test_loss:
            best_test_loss = total_test_loss
            print("Saving the model state dictionary for Epoch: {} with Test loss: {:.8f}".format(epoch + 1, total_test_loss))
            torch.save(model.state_dict(), "model.dth")


    # ***************** sparsity calculation ***************** #
    sparsity_list12 = sparsity_calculator(final_spareness_12)
    sparsity_list23 = sparsity_calculator(final_spareness_23)
    sparsity_list34 = sparsity_calculator(final_spareness_34)
    sparsity_list45 = sparsity_calculator(final_spareness_45)

    print(sparsity_list12)
    print(sparsity_list23)
    print(sparsity_list34)
    print(sparsity_list45)

    average_sparsity = list()
    sparsity12 = list()
    sparsity23 = list()
    sparsity34 = list()
    sparsity45 = list()

    for i in range(no_epochs):
        sparsity12.append(sparsity_list12[i].item())
        sparsity23.append(sparsity_list23[i].item())
        sparsity34.append(sparsity_list34[i].item())
        sparsity45.append(sparsity_list45[i].item())
        average_sparsity.append( (sparsity_list12[i].item() + sparsity_list23[i].item() + sparsity_list34[i].item() + sparsity_list45[i].item()) / 4 )

    # ***************** sparsity calculation ***************** #

    print("average_sparsity:", average_sparsity)

    return test_acc, average_sparsity, sparsity12, sparsity23, sparsity34, sparsity45

# Adadelta

In [ ]:
model_factory('Adadelta')

my_model: Model(
  (linear_1): Linear(in_features=784, out_features=256, bias=True)
  (linear_2): Linear(in_features=256, out_features=256, bias=True)
  (linear_3): Linear(in_features=256, out_features=256, bias=True)
  (linear_4): Linear(in_features=256, out_features=256, bias=True)
  (linear_5): Linear(in_features=256, out_features=10, bias=True)
  (sigmoid12): Sigmoid()
  (sigmoid23): Sigmoid()
  (sigmoid34): Sigmoid()
  (sigmoid45): Sigmoid()
)
my_optimizer: Adadelta (
Parameter Group 0
    eps: 1e-06
    lr: 1.0
    rho: 0.9
    weight_decay: 0
)

Epoch: 1/50, Train Loss: 2.30746806, Test Loss: 2.30202828, Test Accuracy: 0.09580000

Epoch: 2/50, Train Loss: 2.30127321, Test Loss: 2.29988619, Test Accuracy: 0.11350000

Epoch: 3/50, Train Loss: 2.18322140, Test Loss: 1.78346972, Test Accuracy: 0.35970000

Epoch: 4/50, Train Loss: 1.38807292, Test Loss: 1.05196477, Test Accuracy: 0.58940000

Epoch: 5/50, Train Loss: 0.73925256, Test Loss: 0.40720876, Test Accuracy: 0.89020000
Saving 

# Adagrad

In [ ]:
model_factory('Adagrad')

my_model: Model(
  (linear_1): Linear(in_features=784, out_features=256, bias=True)
  (linear_2): Linear(in_features=256, out_features=256, bias=True)
  (linear_3): Linear(in_features=256, out_features=256, bias=True)
  (linear_4): Linear(in_features=256, out_features=256, bias=True)
  (linear_5): Linear(in_features=256, out_features=10, bias=True)
  (sigmoid12): Sigmoid()
  (sigmoid23): Sigmoid()
  (sigmoid34): Sigmoid()
  (sigmoid45): Sigmoid()
)
my_optimizer: Adagrad (
Parameter Group 0
    eps: 1e-10
    initial_accumulator_value: 0
    lr: 0.1
    lr_decay: 0
    weight_decay: 0
)

Epoch: 1/50, Train Loss: 2.41818991, Test Loss: 2.32539597, Test Accuracy: 0.09800000

Epoch: 2/50, Train Loss: 2.32925109, Test Loss: 2.33124026, Test Accuracy: 0.09740000

Epoch: 3/50, Train Loss: 2.32166391, Test Loss: 2.32314413, Test Accuracy: 0.11350000

Epoch: 4/50, Train Loss: 2.31882392, Test Loss: 2.32085728, Test Accuracy: 0.09800000

Epoch: 5/50, Train Loss: 2.31751982, Test Loss: 2.32146739

# SGD

In [ ]:
model_factory('SGD')

my_model: Model(
  (linear_1): Linear(in_features=784, out_features=256, bias=True)
  (linear_2): Linear(in_features=256, out_features=256, bias=True)
  (linear_3): Linear(in_features=256, out_features=256, bias=True)
  (linear_4): Linear(in_features=256, out_features=256, bias=True)
  (linear_5): Linear(in_features=256, out_features=10, bias=True)
  (sigmoid12): Sigmoid()
  (sigmoid23): Sigmoid()
  (sigmoid34): Sigmoid()
  (sigmoid45): Sigmoid()
)
my_optimizer: SGD (
Parameter Group 0
    dampening: 0
    lr: 0.1
    momentum: 0
    nesterov: False
    weight_decay: 0
)

Epoch: 1/50, Train Loss: 2.32315335, Test Loss: 2.30903592, Test Accuracy: 0.09820000

Epoch: 2/50, Train Loss: 2.30906892, Test Loss: 2.30911062, Test Accuracy: 0.10320000

Epoch: 3/50, Train Loss: 2.30566494, Test Loss: 2.30473550, Test Accuracy: 0.11350000

Epoch: 4/50, Train Loss: 2.30376772, Test Loss: 2.30548496, Test Accuracy: 0.10100000

Epoch: 5/50, Train Loss: 2.30315410, Test Loss: 2.30266627, Test Accuracy

# Adam

In [ ]:
model_factory('Adam')

my_model: Model(
  (linear_1): Linear(in_features=784, out_features=256, bias=True)
  (linear_2): Linear(in_features=256, out_features=256, bias=True)
  (linear_3): Linear(in_features=256, out_features=256, bias=True)
  (linear_4): Linear(in_features=256, out_features=256, bias=True)
  (linear_5): Linear(in_features=256, out_features=10, bias=True)
  (sigmoid12): Sigmoid()
  (sigmoid23): Sigmoid()
  (sigmoid34): Sigmoid()
  (sigmoid45): Sigmoid()
)
my_optimizer: Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.001
    weight_decay: 0
)

Epoch: 1/50, Train Loss: 0.63060463, Test Loss: 0.26807362, Test Accuracy: 0.92430000
Saving the model state dictionary for Epoch: 1 with Test loss: 0.26807362

Epoch: 2/50, Train Loss: 0.20602424, Test Loss: 0.17091360, Test Accuracy: 0.95030000
Saving the model state dictionary for Epoch: 2 with Test loss: 0.17091360

Epoch: 3/50, Train Loss: 0.13963422, Test Loss: 0.13862531, Test Accuracy: 0.96050000
Savin